In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pysrim import dose, Ion, Layer

# Useful layer definitions

In [2]:
mils_to_mm = 0.0254        # 1 mil = 0.0254 mm
mm_to_angs = 1e7           # 1 mm = 10,000,000 Å (angstroms)

al_layer = Layer(
    elements={
        'Al': {
            'stoich':   1.0,
            'E_d':     25.0, # Displacement energy [eV], common default
            'lattice':  0.0, # Lattice binding energy [eV]
            'surface': 3.36  # Surface binding energy [eV]
        }
    },
    density=2.702,
    width=3.0 * mils_to_mm * mm_to_angs,
    name="Aluminium"
)

air_layer = Layer(
    elements={
        'N': {'stoich': 0.755},
        'O': {'stoich': 0.232},
        'Ar': {'stoich': 0.013}
    },
    density=0.00125,
    width=80.0 * mm_to_angs,
    phase=1,  # gas
    name = "Air gas"
)

cu_layer = Layer(
    elements={
        'Cu': {'stoich': 1.00},
    },
    density=8.96,
    width=0.46 * mm_to_angs,
    name="Copper"
)

kovar_layer = Layer(
    elements={
        'Fe': {'stoich': 0.534912},
        'Ni': {'stoich': 0.293292},
        'Co': {'stoich': 0.171795},
    },
    density=8.36,
    width=0.9 * mm_to_angs,
    name="Kovar layer"
)

# Example: CSDA energy and range

In [ ]:
ion = Ion('H', energy=25.0e6)
layer = kovar_layer

res_E, range_in, range_out = dose.csda_energy_and_range(ion, layer)
print(f"Residual energy after traversing through {layer.name}: {res_E:.2f} MeV")
print(f"Range in the material: {range_in:.2f} mm")
print(f"Range left after traversing the material: {range_out*1e3:.2f} um")

In [ ]:
depths_mm = np.linspace(0,1.5,100)
residual_energies, range_in, range_out = dose.csda_energy_and_range(ion, layer, depths_mm)
plt.plot(depths_mm, residual_energies)
plt.axvline(x=range_in, ls='--', c='k')
plt.xlabel("Depth in Al (mm)")
plt.ylabel("Proton Energy (MeV)")
plt.title("Proton Energy vs. Depth in Aluminum")
plt.grid(True)
plt.show()

# Example: Ion traversing through layer stack: FPGA

In [ ]:
ion = Ion('H', energy=45.0e6)

stack = [al_layer, air_layer, cu_layer]

E, vE, range_out = dose.energy_profile_through_stack(ion, stack, verbose=True)
print(f"Residual energy after traversing through stack:: {E:.2f} MeV")
print(f"Range left in silicon after traversing the stack: {range_out*1e3:.2f} um")

## Granular sampling

In [ ]:
final_energy, energy_profile, range_left = dose.energy_profile_through_stack(
    ion,
    stack,
    verbose=True,
    granular=30
)

print(f"\nFinal energy: {final_energy:.2f} MeV")
print(f"Range left in silicon: {range_left:.2f} mm")

In [ ]:
dose.plot_energy_profile(energy_profile, stack)

# Example: dose calculation: laser diode inside kovar package

In [ ]:
ion = Ion('H', energy=25.0e6)

stack = [al_layer, air_layer, kovar_layer]

E, vE, range_out = dose.residual_energy_through_stack(ion, stack, verbose=True)
print(f"Residual energy after traversing through stack:: {E:.2f} MeV")
print(f"Range after the material: {range_out*1e3:.2f} um")

In [ ]:
residual_E, ion_range_mm, tid = dose.compute_tid_silicon(ion, stack, 2e7*300, verbose=True)